# Assignment 11  
Implement Hidden Markov Model with supervised training and Viterbi algorithm for finding the most probable sequence of hidden states. Use [Laplace smoothing](https://en.wikipedia.org/wiki/Additive_smoothing) for estimation of probabilities. Apply the developed model to the problems:
* part of speech tagging
* spelling correction

In [1]:
import numpy as np
from scipy import sparse
from tqdm import tqdm
from IPython.display import Image
from collections import defaultdict

HMM model for 3-grams:

$P(x_1, .., x_T, y_1, .., y_T, y_{T+1}) = \prod_{t=1}^{T+1} q(y_t | y_{t-2}, y_{t-1}) \prod_{t=1}^T e(x_t | y_t)$

$x_1, .., x_T$ - sequence of observed states of length T  
$y_1, .., y_T$ - sequence of hidden states of length T  
$q(i | u, v) = \frac {count(u, v, i)} {count(u, v)} $ - transition probability   
$e(x_k | i) = \frac {count(i, x_k)} {count(i)}$ - emission probability  
$A_{i,j} = A_{(u,v), j} = q(i | u, v)$ - transition matrix  
$B_{j,k} = e(x_k | j)$ - emission matrix  

We assume, that $y_{T+1} = TERM$, and $y_0 = y_{-1} = START$

In [2]:
class HMM:
    START = '*'
    TERM = '$'
    REST = '$REST$' # to deal with observed states who have never appeared in train dataset.
        
    def cond_idx(self, u, v):
        return u + v*self.h_dim
        
    def fit(self, X, y):
        """
        X - list of lists, observed states
        y - list of lists, hidden states
        estimate elements of A, B matrices from train sequence. 
        """
        
        assert(len(X) == len(y))
        
        #######################
        # YOUR CODE HERE
        # self.hidden_idx2state = # list of unique hidden states in train sequence + [START, TERM]
        # self.hidden_states = # from state name to state index in hidden_idx2state
        # self.h_dim = # number of hidden states
        # self.observed_idx2state = # lisf of unique observed states in train sequence + [REST]
        # self.observed_states = # from state name to state index in hidden_idx2state
        # self.o_dim = # number of observed states       
        
        self.hidden_idx2state = list(set([word for sublist in y for word in sublist])) + [self.START, self.TERM] # list of unique hidden states in train sequence + [START, TERM]
        self.hidden_states = dict(map(lambda x: (x[1], x[0]), enumerate(self.hidden_idx2state))) # from state name to state index in hidden_idx2state
        self.h_dim = len(self.hidden_idx2state) # number of hidden states
        
        self.observed_idx2state = list(set([word for sublist in X for word in sublist])) + [self.REST] # lisf of unique observed states in train sequence + [REST]
        self.observed_states = dict(map(lambda x: (x[1], x[0]), enumerate(self.observed_idx2state))) # from state name to state index in hidden_idx2state
        self.o_dim = len(self.observed_idx2state) # number of observed states
        #######################
        
        #######################
        # estimate emission matrix
        # YOUR CODE HERE
        # self.B = sparse csr matrix of shape (h_dim, o_dim)
        
        self.B = np.zeros((self.h_dim, self.o_dim))
        
        for i in range(len(X)):
            for o, h in zip(X[i], y[i]):
                self.B[self.h_state(h), self.o_state(o)] += 1
        #######################
        
        
        self.B_rowsum = np.ravel(self.B.sum(axis=1))
        
        
        ########################
        # transition matrix
        # YOUR CODE HERE
        # self.A = dense matrix of shape (h_dim **2, h_dim)
        # remember about padding for sequence of hidden states, eg {a, b} -> {START, START, a, b, TERM}
        
        self.A = np.zeros((self.h_dim ** 2, self.h_dim))
        
        for i in range(len(y)):
            tmp = [self.START] * 2
            tmp.extend(y[i])
            tmp.append(self.TERM)
            for word_i in range(len(tmp) - 2):
                u = self.h_state(tmp[word_i])
                v = self.h_state(tmp[word_i + 1])
                j = self.h_state(tmp[word_i + 2])
                self.A[self.cond_idx(u, v), j] += 1
        ########################
        
        self.A_rowsum = np.ravel(self.A.sum(axis=1))
        
        return self
    
    def tr_prob(self, i, j, alpha=0.1):
        """
        A_ij = q(j | i) = q(j| u, v) with Laplace smoothing
        """
        ########################
        # YOUR CODE HERE
        # result = smoothed probability

        result = (self.A[i, j] + alpha) / (self.A_rowsum[i] + (alpha * self.h_dim))
#         result = (self.A[i, j] + alpha) / (self.A_rowsum[i] + (alpha * self.A.shape[1]))
        ########################
        return result
    
    def em_prob(self, j, k, alpha=0.1):
        """
        B_jk = e(x_k| j) with Laplace smoothing
        """
        ########################
        # YOUR CODE HERE
        # result = smoothed probability
        
        result = (self.B[j, k] + alpha) / (self.B_rowsum[j] + (alpha * self.o_dim))
#         result = (self.B[j, k] + alpha) / (self.B_rowsum[j] + (alpha * self.B.shape[1]))
        ########################
        return result
        
    def o_state(self, x):
        """
        return index of observed state
        """
        return self.observed_states.get(x, self.observed_states[self.REST])
    
    def h_state(self, x):
        """
        return index of hidden state
        """
        return self.hidden_states.get(x)
    
    def predict(self, X):
        """
        Predict the most probable sequence of hidden states for every sequence of observed states
        X - list of lists
        """
        y_pred = [self._viterbi(seq) for seq in tqdm(X)]
        return y_pred 
            
    def _viterbi(self, X, alpha=0.1):
        """
        X - list of observables
        product of probabilities usually is not numerically stable
        remember, that log(ab) = log(a) + log(b) and argmax[log(f(x))] = argmax[f(x)]
        
        """   
        T = len(X)
        
        # pi[t, u, v] - max probability for any state sequence ending with x_t = v and x_{t-1} = u.
        pi = np.zeros((T + 1, self.h_dim, self.h_dim))
        # backpointers, bp[t, u, v] = argmax probability for any state sequence ending with x_t = v and x_{t-1} = u.
        bp = np.zeros((T + 1, self.h_dim, self.h_dim), dtype=np.int)
        
        ###################
        # fill tables pi and bp
        # pi[t, u, v] =    max_{w} [ pi[t-1, w, u] * q(v| w, u) * e(x_k| v) ]
        # bp[t, u, v] = argmax_{w} [ pi[t-1, w, u] * q(v| w, u) * e(x_k| v) ]
        # YOUR CODE HERE 
#         for k in range(1, T + 1):
#             xk = self.o_state(X[k-1])
            
#             for v in range(self.h_dim):
#                 log_b_smoothed = 
#                 for u in range(self.h_dim): 
#                     r = np.zeros(self.h_dim)
#                     for w in range(self.h_dim):
#                         log_a_smoothed = 
#                         r[w] = 
#                     bp[k, u, v] = np.argmax(r)
#                     pi[k, u, v] = np.max(r)
        
        for k in range(1, T + 1):
            xk = self.o_state(X[k-1])
            
            for v in range(self.h_dim):
                log_b_smoothed = np.log(self.em_prob(v, xk, alpha))

                for u in range(self.h_dim): 
                    r = np.zeros(self.h_dim)
                    for w in range(self.h_dim):
                        log_a_smoothed = np.log(self.tr_prob(self.cond_idx(w, u), v, alpha))
                        r[w] = pi[k-1, w, u] + log_b_smoothed + log_a_smoothed
                    bp[k, u, v] = np.argmax(r)
                    pi[k, u, v] = np.max(r)
        ###################
                    
        term_idx = self.hidden_states[self.TERM]
        
        ###################
        # r(u,v) = pi[T, u, v] * q(TERM | u, v)
        # find argmax_{u, v} r(u, v)
        # YOUR CODE HERE
        # u, v = 
        
        r = np.zeros((self.h_dim, self.h_dim))
        for u in range(r.shape[0]):
            for v in range(r.shape[1]):
                r[u, v] = pi[T, u, v] + np.log(self.tr_prob(self.cond_idx(u, v), self.h_state(self.TERM), alpha))
        
        u, v = np.argmax(r) // r.shape[1], np.argmax(r) % r.shape[1]
        ###################

        h_states = [v, u]
        ###################
        # rollback backpointers
        # y_{t-2} = bp[t, y_{t-1}, y_t]
        # y_{t} = bp[t + 2, y_{t+1}, y_{t+2}]
        # h_states is a reversed sequence of hidden states
        # YOUR CODE HERE
        # h_states = 
        
        for t in range(T-2, -1, -1):
            h_states.append(bp[t+2, h_states[-1], h_states[-2]])
        ###################        
            
        return [self.hidden_idx2state[i] for i in reversed(h_states[:T])]

## Problem 1. Part of speech tagging

Build Part-of-Speech tagging model using HMM. Estimate accuracy on test dataset.

In [3]:
import nltk
nltk.download('treebank')
from nltk.corpus import treebank
from sklearn import metrics

[nltk_data] Downloading package treebank to
[nltk_data]     /Users/AntonKarazeev/nltk_data...
[nltk_data]   Unzipping corpora/treebank.zip.


In [4]:
data = treebank.tagged_sents()[:3000]
test_data = treebank.tagged_sents()[3000:3010]

X_train = [[x[0] for x in y] for y in data]
y_train = [[x[1] for x in y] for y in data]

X_test = [[x[0] for x in y] for y in test_data]
y_test = [[x[1] for x in y] for y in test_data]

print('sentence: ', " ".join(X_train[0]))
print('tags: ', " ".join(y_train[0]))

sentence:  Pierre Vinken , 61 years old , will join the board as a nonexecutive director Nov. 29 .
tags:  NNP NNP , CD NNS JJ , MD VB DT NN IN DT JJ NN NNP CD .


In [5]:
def accuracy(y_true, y_pred):       
    y_true = np.concatenate(y_true)
    y_pred = np.concatenate(y_pred)
    return np.mean(y_true == y_pred)

In [6]:
%%time

hh = HMM()
hh.fit(X_train, y_train)
y_pred = hh.predict(X_test[:1])
print(accuracy(y_test[:1], y_pred))

100%|██████████| 1/1 [00:10<00:00, 10.75s/it]

0.75
CPU times: user 11.1 s, sys: 161 ms, total: 11.2 s
Wall time: 11.1 s


Your accuracy must be > 0.74

## Problem 1.2 Vectorized viterbi
Our currrent implementation of Viterbi is too slow. Let's make it vectorized. 

In [19]:
class HmmVectorized(HMM):
    
    def _viterbi(self, X, alpha=0.1):
        """
        Vectorized version of Viterbi. Let's speed up!
        X - list of observables
        """   
        T = len(X)
        
        # One may notice, at every step t we only need pi[t-1, u, v] = pi_prev[u,v] to compute pi[t, u, v] = pi_curr[u,v]
        pi_prev = np.zeros((self.h_dim, self.h_dim))
        
        # backpointers
        bp = np.zeros((T + 1, self.h_dim, self.h_dim), dtype=np.int)
        
        a_rowsum = self.A_rowsum.reshape(self.h_dim, self.h_dim)
        
        ###################
        # fill pi and bp
        # pi_curr[u, v] = max_{w} [ pi_prev[w, u] * q(v| w, u) * e(x_k| v) ]
        # bp[t, u, v] = argmax_{w} [ pi_prev[w, u] * q(v| w, u) * e(x_k| v) ]
        # don't use tr_prob() and em_prob(), apply laplace smoothing directly here
        # YOUR CODE HERE
#         for k in range(1, T + 1):            
#             xk = self.o_state(X[k-1])
#             pi_curr = np.zeros_like(pi_prev)
            
#             for v in range(self.h_dim):
#                 log_b_smoothed = 
#                 a = self.A[:, v].reshape(self.h_dim, self.h_dim)
#                 log_a_smoothed = 
#                 r =  
#                 bp[k, :, v] = np.argmax(r, axis=1)
#                 pi_curr[:, v] = np.max(r, axis=1)
                    
#             pi_prev = pi_curr

        for k in range(1, T + 1):            
            xk = self.o_state(X[k-1])
            pi_curr = np.zeros_like(pi_prev)
            
            for v in range(self.h_dim):
                a = self.A[:, v].reshape(self.h_dim, self.h_dim)
                
                log_b_smoothed = np.log((self.B[v, xk] + alpha) / (self.B_rowsum[v] + (alpha * self.o_dim)))
                log_a_smoothed = np.log((a + alpha) / (a_rowsum + (alpha * self.h_dim)))
                r = pi_prev.T + log_a_smoothed + log_b_smoothed
                
                bp[k, :, v] = np.argmax(r, axis=1)
                pi_curr[:, v] = np.max(r, axis=1)
                    
            pi_prev = pi_curr
        ###################
        
        term_idx = self.hidden_states[self.TERM]
        
        ###################
        # r(u,v) = pi[T, u, v] * q(TERM | u, v)
        # find argmax_{u, v} r(u, v)
        # express r(u,v) as matrix additions
        # YOUR CODE HERE
        # u, v = 

        a = self.A[:, self.h_state(self.TERM)].reshape(self.h_dim, self.h_dim)
        log_a_smoothed = np.log((a + alpha) / (a_rowsum + (alpha * self.h_dim)))
        r = pi_prev + log_a_smoothed
        
        u, v = np.argmax(r) // r.shape[1], np.argmax(r) % r.shape[1]
        ###################
        
        h_states = [v, u]
        ###################
        # rollback backpointers
        # y_{t-2} = bp[t, y_{t-1}, y_t]
        # h_states is a reversed sequence of hidden states
        # YOUR CODE HERE
        # h_states = 
        
        for t in range(T-2, -1, -1):
            h_states.append(bp[t+2, h_states[-1], h_states[-2]])
        ###################
        
        return [self.hidden_idx2state[i] for i in reversed(h_states[:T])]

Let's take a larger test subset

In [20]:
data = treebank.tagged_sents()[:3000]
test_data = treebank.tagged_sents()[3000:3300]

X_train = [[x[0] for x in y] for y in data]
y_train = [[x[1] for x in y] for y in data]

X_test = [[x[0] for x in y] for y in test_data]
y_test = [[x[1] for x in y] for y in test_data]

print('sentence: ', " ".join(X_train[0]))
print('tags: ', " ".join(y_train[0]))

sentence:  Pierre Vinken , 61 years old , will join the board as a nonexecutive director Nov. 29 .
tags:  NNP NNP , CD NNS JJ , MD VB DT NN IN DT JJ NN NNP CD .


In [21]:
%%time

hh = HmmVectorized().fit(X_train, y_train)
y_pred = hh.predict(X_test)
print(accuracy(y_test, y_pred))


100%|██████████| 300/300 [00:34<00:00,  8.62it/s]

0.900133155792
CPU times: user 33.7 s, sys: 488 ms, total: 34.2 s
Wall time: 35.2 s


Your accuracy must be > 0.84 

## Problem 2. Spelling correction

Given data of true_char corrupted\_char build spelling correction model using HMM.    
There are 2 datatsets (spelling10.txt, spelling20.txt) with 10% and 20% corruption probability respectively.  
Each dataset contains 30556 words. Use first 28000 for training and the rest for testing. Estimate accuracy on the test subset.  


In [ ]:
class HMMSpelling:
    START = '*'
    TERM = '$'
    REST = '$REST$'
        
    def fit(self, X, y):
        """
        X - list of lists, observed states
        y - list of lists, hidden states
        estimate elements of A, B matrices from train sequence. 
        """
        
        assert(len(X) == len(y))
        
        self.hidden_idx2state = list(set([word for sublist in y for word in sublist])) + [self.START, self.TERM]
        self.hidden_states = dict(map(lambda x: (x[1], x[0]), enumerate(self.hidden_idx2state)))
        self.h_dim = len(self.hidden_idx2state)
        
        self.observed_idx2state = list(set([word for sublist in X for word in sublist])) + [self.REST]
        self.observed_states = dict(map(lambda x: (x[1], x[0]), enumerate(self.observed_idx2state)))
        self.o_dim = len(self.observed_idx2state)
        
        self.B = np.zeros((self.h_dim, self.o_dim))
        for i in range(len(X)):
            for o, h in zip(X[i], y[i]):
                self.B[self.h_state(h), self.o_state(o)] += 1
        
        self.B_rowsum = np.ravel(self.B.sum(axis=1))
        
        
        self.A = np.zeros((self.h_dim, self.h_dim))
        for i in range(len(y)):
            tmp = [self.START]
            tmp.extend(y[i])
            tmp.append(self.TERM)
            for word_i in range(len(tmp) - 1):
                u = self.h_state(tmp[word_i])
                j = self.h_state(tmp[word_i + 1])
                self.A[u, j] += 1
        
        self.A_rowsum = np.ravel(self.A.sum(axis=1))
        
        return self
    
    def tr_prob(self, i, j, alpha=0.1):
        """
        A_ij = q(j | i) with Laplace smoothing
        """
        result = (self.A[i, j] + alpha) / (self.A_rowsum[i] + (alpha * self.h_dim))
        return result
    
    def em_prob(self, j, k, alpha=0.1):
        """
        B_jk = e(x_k| j) with Laplace smoothing
        """
        result = (self.B[j, k] + alpha) / (self.B_rowsum[j] + (alpha * self.o_dim))
        return result
        
    def o_state(self, x):
        """
        return index of observed state
        """
        return self.observed_states.get(x, self.observed_states[self.REST])
    
    def h_state(self, x):
        """
        return index of hidden state
        """
        return self.hidden_states.get(x)
    
    def predict(self, X):
        """
        Predict the most probable sequence of hidden states for every sequence of observed states
        X - list of lists
        """
        y_pred = [self._viterbi(seq) for seq in tqdm(X)]
        return y_pred 
            
    def _viterbi(self, X, alpha=0.1):
        """
        Vectorized version of Viterbi. Let's speed up!
        X - list of observables
        """   
        T = len(X)

        pi_prev = np.zeros((self.h_dim, self.h_dim))

        bp = np.zeros((T + 1, self.h_dim, self.h_dim), dtype=np.int)
        
        a_rowsum = self.A_rowsum.reshape(self.h_dim, self.h_dim)

        for k in range(1, T + 1):            
            xk = self.o_state(X[k-1])
            pi_curr = np.zeros_like(pi_prev)
            
            for v in range(self.h_dim):
                a = self.A[:, v].reshape(self.h_dim, self.h_dim)
                
                log_b_smoothed = np.log((self.B[v, xk] + alpha) / (self.B_rowsum[v] + (alpha * self.o_dim)))
                log_a_smoothed = np.log((a + alpha) / (a_rowsum + (alpha * self.h_dim)))
                r = pi_prev.T + log_a_smoothed + log_b_smoothed
                
                bp[k, :, v] = np.argmax(r, axis=1)
                pi_curr[:, v] = np.max(r, axis=1)
                    
            pi_prev = pi_curr

        a = self.A[:, self.h_state(self.TERM)].reshape(self.h_dim, self.h_dim)
        log_a_smoothed = np.log((a + alpha) / (a_rowsum + (alpha * self.h_dim)))
        r = pi_prev + log_a_smoothed
        
        u, v = np.argmax(r) // r.shape[1], np.argmax(r) % r.shape[1]
        
        h_states = [v, u]
        
        for t in range(T-2, -1, -1):
            h_states.append(bp[t+2, h_states[-1], h_states[-2]])
        
        return [self.hidden_idx2state[i] for i in reversed(h_states[:T])]

In [116]:
with open('data/spelling10.txt', 'r') as f:
    data10 = f.readlines()
    data10 = list(map(lambda x: x.strip().split(), data10))

indexes = [-1]
indexes.extend([i for i, x in enumerate(data10) if x == ['_', '_']])
indexes.append(len(data10))

tmp = []
for i in range(1, len(indexes)):
    tmp.append(data10[indexes[i-1]+1:indexes[i]])
data10 = tmp
data10[:2]

[[['b', 'b'], ['y', 'y']],
 [['t', 't'], ['h', 'h'], ['e', 'e'], ['i', 'i'], ['r', 'r']]]

In [117]:
num_train = 28000
X_train = [[ch[1] for ch in word] for word in data10[:num_train]]
y_train = [[ch[0] for ch in word] for word in data10[:num_train]]

X_test = [[ch[1] for ch in word] for word in data10[num_train:]]
y_test = [[ch[0] for ch in word] for word in data10[num_train:]]

In [118]:
print('word: ', " ".join(X_train[-11]))
print('correct word: ', " ".join(y_train[-11]))

word:  u n f e r i o r i t y
correct word:  i n f e r i o r i t y


In [120]:
%%time

hh = HmmVectorized().fit(X_train, y_train)
y_pred = hh.predict(X_test)
print(accuracy(y_test, y_pred))


100%|██████████| 2558/2558 [00:24<00:00, 105.93it/s]

0.87374416581
CPU times: user 23.7 s, sys: 549 ms, total: 24.2 s
Wall time: 24.8 s


You should get > 93% accuracy (+3%) on spelling10 dataset

In [122]:
with open('data/spelling20.txt', 'r') as f:
    data20 = f.readlines()
    data20 = list(map(lambda x: x.strip().split(), data20))

indexes = [-1]
indexes.extend([i for i, x in enumerate(data20) if x == ['_', '_']])
indexes.append(len(data10))

tmp = []
for i in range(1, len(indexes)):
    tmp.append(data20[indexes[i-1]+1:indexes[i]])
data20 = tmp
data20[:2]

[[['b', 'b'], ['y', 'y']],
 [['t', 'y'], ['h', 'j'], ['e', 'w'], ['i', 'i'], ['r', 'r']]]

In [126]:
num_train = 28000
X_train = [[ch[1] for ch in word] for word in data20[:num_train]]
y_train = [[ch[0] for ch in word] for word in data20[:num_train]]

X_test = [[ch[1] for ch in word] for word in data20[num_train:]]
y_test = [[ch[0] for ch in word] for word in data20[num_train:]]

In [127]:
print('word: ', " ".join(X_train[-11]))
print('correct word: ', " ".join(y_train[-11]))

word:  i n f e r i o f i t y
correct word:  i n f e r i o r i t y


In [128]:
%%time

hh = HmmVectorized().fit(X_train, y_train)
y_pred = hh.predict(X_test)
print(accuracy(y_test, y_pred))


100%|██████████| 2558/2558 [00:20<00:00, 127.04it/s]


0.809625583788
CPU times: user 20.5 s, sys: 253 ms, total: 20.7 s
Wall time: 20.8 s


You should get > 89% accuracy (+9%) on spelling20 dataset